In [9]:
import os
import string

def get_filtered_data(rootDir):
    ''' Return filtered data from root folder as array of strings and 
    the corresponding paths to the respective files
    '''
    allDirs = []
    allFiles = []
    files_txt = []
    data = []
    dataFiltered = []
    filesFiltered = []
    
    translator = str.maketrans('', '', string.punctuation)

    for root, dirs, files in os.walk(rootDir, topdown=False):
        for name in dirs:
            allDirs.append(os.path.join(root, name))

    for dP in allDirs:
        onlyFiles = [f for f in os.listdir(dP) if os.path.isfile(os.path.join(dP, f))]
        for oF in onlyFiles: allFiles.append(os.path.join(dP, oF))

    for aF in allFiles:
        #with open(aF, 'r', encoding = "ISO-8859-1") as newFile: 
        with open(aF, 'r', encoding = "utf-8", errors='replace') as newFile: 
            data.append(newFile.read())
            
    for ind, fileData in enumerate(data):
        filt = [d == '\x0c' for d in fileData]
        if sum(filt) < len(filt):
            s = fileData.lower()
            s = s.translate(translator)
            dataFiltered.append(s)
            filesFiltered.append(allFiles[ind])
            
    return (dataFiltered, filesFiltered)


def document_to_words(doc):
    doc = doc.split('\n')
    doc_short = []
    doc_words = []
    doc_words_short = []
    for d in doc:
        if len(d)>1: doc_short.append(d)
    for d in doc_short:
        doc_words += d.split(' ')
    for d in doc_words:
        if len(d)>1: doc_words_short.append(d)
    return doc_words_short

def remove_stopwords(tokens):
    '''Remove stop words from an array of tokens'''
    
    from nltk.corpus import stopwords
    #nltk.download('stopwords')
    #stopWords = set(stopwords.words('english'))
    stopWords = ['the', 'to', '-', 'pr', 'der', 'is', 'of', 'die', 'in', 'and', 'und', '–', '•', '✔', '●', 'a']
    
    tokens_filt = []
    for gT in tokens:
        if gT not in stopWords: tokens_filt.append(gT)
            
    return tokens_filt


def tokenize(data):
    '''Extract different token arrays for every single files in data (>>tokens)
    and extract ONE single token array for all files (>>globalTokens)'''
    tokens = []
    for d in data: 
        tokens.append(nltk.word_tokenize(d))
    return tokens

def calc_document_tf(tokens):
    '''Compute tf's for every file 
        -> dict: tf[indFile]['word'] = tf 
    '''
    count = Counter(tokens)
    totalCount = sum(count.values())
    tf_doc = {}
    for c in count:
        tf_doc[c] = count[c] / totalCount
    return tf_doc

def calc_idf(term, tokens_filtered):
    '''Compute idf's'''
    no_of_doc = 0
    conn = []
    # Count number of documents, in which the term occurs
    for ind, t in enumerate(tokens_filtered):
        if (term in t): 
            no_of_doc += 1
            conn.append(ind)
    if (no_of_doc > 0): return (log(len(tokens_filtered) / no_of_doc), conn)
    else: return (None, conn)


In [10]:
import nltk
import string
import os

# Loading the data files
rootDir = '../docs_txt'
print("Loading data...")
(data, fileDirs) = get_filtered_data(rootDir)
print("Tokenizing...")
tokens = tokenize(data)
print("Removing stopwords...")
tokens_filtered = []
for t in tokens: 
    tokens_filtered.append(remove_stopwords(t))

Loading data...
Tokenizing...
Removing stopwords...


In [4]:
from collections import Counter
from math import log

print("Calculate tf's")
tf = calc_document_tf(tokens_filtered[0])
# Calculate tf's for every word in every file
TF = []
for t in tokens_filtered:
    tf = calc_document_tf(t)
    TF.append(tf)

print("Calculate idf's", flush=True)
print("----------|")
IDF = {}
for ind, t_glob in enumerate(globalTokens_filtered):
    if (ind % int(0.1*len(globalTokens_filtered)) == 0): print("#", flush=True, end='')
    IDF[t_glob] = calc_idf(t_glob, tokens_filtered)

word_conn = {}
for word in IDF:
    conn = IDF[word][1]
    tfs = [TF[c][word] for c in conn]
    idf = IDF[word][0]
    word_conn[word] = (conn, tfs, idf)


print("\nDONE!")


Calculate tf's
Calculate idf's
----------|
#

KeyboardInterrupt: 

In [51]:
from collections import defaultdict
token2files = defaultdict(set)
filentoken2occ = defaultdict(int)
token2occ = defaultdict(int)
for i, tokens in enumerate(tokens_filtered):
    for t in tokens:
        if i not in token2files[t]:
            token2files[t].add(i)
        filentoken2occ[i, t] += 1
        token2occ[t] += 1

    

In [52]:
tokens_filtered_test = set()
for t in tokens_filtered:
    tokens_filtered_test.update(t)
len(tokens_filtered_test)

67074

In [53]:
len(filentoken2occ)

207769

In [54]:
len(token2occ)

67074

In [62]:
import math
n_docs = len(tokens_filtered)
idf = {}
for token, files in token2files.items():
    token_file_n = len(files)
    idf_token = math.log(n_docs / token_file_n)
    idf[token] = idf_token

In [63]:
filentoken2tfidf = defaultdict(int)
for file, token in filentoken2occ.keys():
    doc_len = len(tokens_filtered[file])
    token_occ_doc = filentoken2occ[file, token]
    filentoken2tfidf[file, token] = token_occ_doc / doc_len * idf[token]

In [ ]:
def search(query, filentoken2tf, idf, token2files):
    query_tok = word_tokenize(query)
    tok2tfidf
    for tok in query_tok:
        
    
    

In [64]:
filentoken2tfidf

defaultdict(int,
            {(0, 'mtvess'): 0.01540623778241738,
             (0, 'rov341'): 0.007317962946648255,
             (0, 'project'): 0.01782160364585279,
             (0, 'm44mecc110'): 0.013592113328427965,
             (0, 'item'): 0.006017466998539713,
             (0, 'sn'): 0.010081374811585874,
             (0, '3268'): 0.01163980676761266,
             (0, 'date'): 0.006892319644153594,
             (0, '17012011'): 0.021953888839944764,
             (0, 'studio'): 0.033893723121318234,
             (0, 'tecnico'): 0.012095566385563607,
             (0, 'calculation'): 0.014942584829436468,
             (0, 'code'): 0.008712753999923824,
             (0, 'ad'): 0.007712166579546131,
             (0, '2000merkblatt'): 0.018102329394340422,
             (0, 'updales'): 0.0007703118891208689,
             (0, 'march'): 0.009315113764866295,
             (0, '2009'): 0.009189321132325784,
             (0, 'acv341'): 0.00038515594456043446,
             (0, 'ltem'): 0.005

In [66]:
list(enumerate(fileDirs))

[(0,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Calcolation.pdf.txt'),
 (1,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Weld Map and WPS.pdf.txt'),
 (2,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Non Destructive Examination.pdf.txt'),
 (3,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Marking Map & Mill Test Report SN 3268.pdf.txt'),
 (4,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Nameplate.pdf.txt'),
 (5,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Deckblatt.pdf.txt'),
 (6,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Welders Qualification.pdf.txt'),
 (7,
  '../docs_txt/unklar/Gesamtdokumente/Bedienungsanleitung Mischsilo/8. Quality Manual/Non Destructive Operators level.pdf.txt'),
 (8,
  '../doc

In [73]:
filentoken2tfidf[212, 'zimmerlin']

0.2648021296435674

In [74]:
filentoken2tfidf[212, 'lochkreis']

0.2648021296435674

In [75]:
filentoken2tfidf[212, 'schrauben']

0.09307754890143875

In [78]:
filentoken2tfidf[212, 'd=230']

0